# **Wine Quality Prediction**

## Introduction

Wine quality is often judged by trained tasters, which can be subjective and time consuming. By applying supervised machine learning to the chemical properties of wine, we aim to build a model that can predict wine quality. Instead of using the full range of quality scores, we simplified things by splitting wines into two groups: **Standard** (quality score 6 or below) and **Premium** (quality score above 6).
This makes it easier for winemakers to spot high-quality wines and helps consumers choose better options, all with the help of data. The model can make quality control easier for producers and help consumers find better wine choices with greater consistency and accuracy.

## Data Preparation: Merging & Cleaning Wine Data

We first merge red and white wine datasets, standardize the numeric features, and save the cleaned version as `cleaned_wine_data_with_scaling.csv` for use in Spark and ML models.


In [1]:
# Load libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [2]:
# Read red and white wine datasets
red_wine = pd.read_csv("winequality-red.csv", sep=';')
white_wine = pd.read_csv("winequality-white.csv", sep=';')

In [3]:
# Add wine_type column with string labels: 'red' for red wine, 'white' for white wine
red_wine["wine_type"] = "red"
white_wine["wine_type"] = "white"

In [4]:
red_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red


In [5]:
white_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,white
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,white
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,white
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white


In [6]:
print(red_wine.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
 12  wine_type             1599 non-null   object 
dtypes: float64(11), int64(1), object(1)
memory usage: 162.5+ KB
None


In [7]:
print(white_wine.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
 12  wine_type             4898 non-null   object 
dtypes: float64(11), int64(1), object(1)
memory usage: 497.6+ KB
None


In [8]:
# Combine red and white wine DataFrames
combined_wine = pd.concat([red_wine, white_wine], ignore_index=True)

In [9]:
# Check for missing values (optional)
print("Missing values before cleaning:")
print(combined_wine.isnull().sum())

Missing values before cleaning:
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
wine_type               0
dtype: int64


In [10]:
# Define which columns get which type of scaling
standardize_cols = ["fixed acidity", "volatile acidity", "citric acid"]
normalize_cols = [
    "residual sugar", "chlorides", "free sulfur dioxide",
    "total sulfur dioxide", "density", "pH",
    "sulphates", "alcohol"
]

# Apply StandardScaler to standardize
std_scaler = StandardScaler()
standardized = std_scaler.fit_transform(combined_wine[standardize_cols])
standardized_df = pd.DataFrame(standardized, columns=standardize_cols)

# Apply MinMaxScaler to normalize
minmax_scaler = MinMaxScaler()
normalized = minmax_scaler.fit_transform(combined_wine[normalize_cols])
normalized_df = pd.DataFrame(normalized, columns=normalize_cols)

# Combine the scaled data
scaled_df = pd.concat([standardized_df, normalized_df], axis=1)

# Add back 'quality' and 'wine_type'
scaled_df["quality"] = combined_wine["quality"]
scaled_df["wine_type"] = combined_wine["wine_type"]

# Round all numeric columns to 4 decimal places
scaled_df = scaled_df.round(4)

# Sort by quality
scaled_df = scaled_df.sort_values(by="quality", ascending=True)

# Preview
scaled_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type
2472,2.3796,-1.0306,1.0417,0.0123,0.0465,0.0139,0.0622,0.1309,0.1318,0.0337,0.2319,3,white
2339,-0.2432,0.3058,0.5600,0.0613,0.0216,0.0139,0.0300,0.0846,0.4574,0.0843,0.6667,3,white
2633,0.5282,1.8244,0.9729,0.1534,0.3904,0.1111,0.5092,0.2157,0.1163,0.2921,0.1594,3,white
2828,0.8367,-0.0587,0.6976,0.0084,0.0399,0.0590,0.2074,0.0769,0.3721,0.0562,0.6377,3,white
2044,-0.0889,-0.1195,0.0094,0.1595,0.0482,0.0521,0.1382,0.1270,0.4031,0.1011,0.5072,3,white


In [11]:
# Save cleaned data to CSV
scaled_df.to_csv("cleaned_wine_data_with_scaling.csv", index=False)
print("Data cleaning and preprocessing completed.")

Data cleaning and preprocessing completed.



## Analyze Cleaned Wine Data using Spark SQL

We now use Spark to analyze the cleaned wine dataset. This demonstrates working with large datasets using distributed computing tools.

We'll register the data as a temporary SQL table and run queries on:
- Average wine quality by type
- Distribution of alcohol content
- Statistical summary of features


In [12]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("WineQualityAnalysis").getOrCreate()

# Load cleaned wine dataset
df_spark = spark.read.csv("cleaned_wine_data_with_scaling.csv", header=True, inferSchema=True)

# Register temp view for SQL queries
df_spark.createOrReplaceTempView("wine_data")

In [13]:
# Query: Average Wine Quality By Type
print("Average Quality By Wine Type")
avg_quality_spark_df = spark.sql("""
    SELECT wine_type, ROUND(AVG(quality), 2) AS avg_quality
    FROM wine_data
    GROUP BY wine_type
""")
avg_quality_spark_df.show()

Average Quality By Wine Type
+---------+-----------+
|wine_type|avg_quality|
+---------+-----------+
|    white|       5.88|
|      red|       5.64|
+---------+-----------+



On average, white wines had a slightly higher quality score (5.88) than red wines (5.63). While both fall into the Standard category based on our classification system (scores of 6 or lower), white wines are generally closer to the Premium category. This might reflect subtle differences in production techniques or ingredients that push white wines closer to higher quality scores on average.

In [14]:
# Query: Top 5 wines with highest alcohol content
print("Top 5 Wines with Highest Alcohol Content")
top5_alcohol_spark_df = spark.sql("""
    SELECT alcohol, quality, wine_type
    FROM wine_data
    ORDER BY alcohol DESC
    LIMIT 5
""")
top5_alcohol_spark_df.show()

Top 5 Wines with Highest Alcohol Content
+-------+-------+---------+
|alcohol|quality|wine_type|
+-------+-------+---------+
|    1.0|      5|      red|
| 0.8986|      7|    white|
| 0.8768|      7|    white|
| 0.8696|      6|    white|
| 0.8696|      6|      red|
+-------+-------+---------+



Among the top five wines with the highest alcohol content, both red and white wines are represented. Interestingly, the wine with the highest alcohol content of 1.0 is a red wine, but it received a quality score of only 5, placing it in the Standard category. This suggests that while alcohol content can influence quality, it doesn’t directly determine whether a wine is Premium. Other chemical features likely play a bigger role in predicting overall quality.

In [15]:
# Query: Feature Averages by Quality Class (Standard vs. Premium)
print("Feature Averages by Standard vs. Premium Wines")
feature_avg_binary_df = spark.sql("""
    SELECT
        CASE WHEN quality <= 6 THEN 'Standard' ELSE 'Premium' END AS quality_label,
        ROUND(AVG(alcohol), 2) AS avg_alcohol,
        ROUND(AVG(sulphates), 2) AS avg_sulphates,
        ROUND(AVG(pH), 2) AS avg_pH,
        ROUND(AVG(density), 4) AS avg_density
    FROM wine_data
    GROUP BY quality_label
""")
feature_avg_binary_df.show()

Feature Averages by Standard vs. Premium Wines
+-------------+-----------+-------------+------+-----------+
|quality_label|avg_alcohol|avg_sulphates|avg_pH|avg_density|
+-------------+-----------+-------------+------+-----------+
|      Premium|        0.5|         0.18|  0.39|     0.1141|
|     Standard|       0.33|         0.17|  0.38|     0.1541|
+-------------+-----------+-------------+------+-----------+



By grouping wines into Standard and Premium categories, we can see clear differences in chemical properties.
Premium wines tend to have higher alcohol content and lower density, reinforcing those features as key indicators of higher quality. Sulphates and pH levels show smaller differences, but still contribute useful signals.
This grouping aligns directly with how our model was trained, giving a clearer view of how chemistry relates to our classification labels.


## Build a Predictive Model

We'll convert the wine quality scores into two categories to simplify the prediction task:

- **Standard**: Wines with a quality score of 6 or lower  
- **Premium**: Wines with a quality score above 6  

This binary classification helps us focus on identifying top-tier wines more effectively. We then train a Random Forest Classifier and evaluate its performance using accuracy, precision, recall, and F1-score to see how well it can distinguish between Standard and Premium wines based on their chemical features.

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd

In [17]:
# Load data
df = pd.read_csv("cleaned_wine_data_with_scaling.csv")

In [38]:
# Convert quality into categories
def categorize_quality_new(q):
    if q <= 6:
        return 0  # Change 'Standard' to 0
    else:
        return 1  # Change 'Premium' to 1

df['quality_label'] = df['quality'].apply(categorize_quality_new)

# Encode categorical variables
# No need to encode 'quality_label' since it's already numerical
df_encoded = pd.get_dummies(df, columns=['wine_type'], drop_first=False)

# Prepare features and target
# Check if any columns in X contain the string 'Standard'
string_cols = [col for col in df_encoded.columns if df_encoded[col].dtype == 'object' and df_encoded[col].str.contains('Standard').any()]
print(f"Columns containing 'Standard': {string_cols}")

# Remove the problematic column(s)
X = df_encoded.drop(columns=['quality', 'quality_label'] + string_cols)
y = df['quality_label']

Columns containing 'Standard': ['Quality']


In [39]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [40]:
# Train Random Forest Classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [41]:
# Predictions
y_pred = model.predict(X_test)

In [42]:
# Initial Evaluation (Pre-Optimization)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2%}\n")

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Model Accuracy: 89.92%

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      1061
           1       0.83      0.57      0.67       239

    accuracy                           0.90      1300
   macro avg       0.87      0.77      0.81      1300
weighted avg       0.89      0.90      0.89      1300

Confusion Matrix:
[[1033   28]
 [ 103  136]]




## Model Optimization & Comparison

We compare the performance of three models:
- Random Forest
- Logistic Regression
- Support Vector Machine (SVM)

Each is trained with:
- Full feature set
- PCA-reduced feature set

We report:
- Accuracy
- Classification Report
- Confusion Matrix


In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import PCA
import numpy as np

# Reload data
df = pd.read_csv("cleaned_wine_data_with_scaling.csv")
df["quality_label"] = df["quality"].apply(categorize_quality_new)


In [44]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type,quality_label
0,2.3796,-1.0306,1.0417,0.0123,0.0465,0.0139,0.0622,0.1309,0.1318,0.0337,0.2319,3,white,0
1,-0.2432,0.3058,0.5600,0.0613,0.0216,0.0139,0.0300,0.0846,0.4574,0.0843,0.6667,3,white,0
2,0.5282,1.8244,0.9729,0.1534,0.3904,0.1111,0.5092,0.2157,0.1163,0.2921,0.1594,3,white,0
3,0.8367,-0.0587,0.6976,0.0084,0.0399,0.0590,0.2074,0.0769,0.3721,0.0562,0.6377,3,white,0
4,-0.0889,-0.1195,0.0094,0.1595,0.0482,0.0521,0.1382,0.1270,0.4031,0.1011,0.5072,3,white,0


In [45]:
# Features and target
X = df.drop(columns=["quality", "wine_type", "quality_label"])
y = df["quality_label"]

In [46]:
# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [47]:
# Initialize PCA
pca = PCA(n_components=5)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [48]:
# Initialize models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=2000, solver='liblinear'),
    "SVM": SVC(class_weight='balanced')
}

In [49]:
# Store results
results = []

In [50]:
for name, model in models.items():
    # Train and evaluate without PCA
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results.append((name, "Models With No PCA", acc))
    print(f"{name} - No PCA")
    print(f"Accuracy: {acc:.2%}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("Classification Report:")
    print(classification_report(y_test, y_pred, zero_division=0))

Random Forest - No PCA
Accuracy: 89.46%
Confusion Matrix:
[[1026   35]
 [ 102  137]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      1061
           1       0.80      0.57      0.67       239

    accuracy                           0.89      1300
   macro avg       0.85      0.77      0.80      1300
weighted avg       0.89      0.89      0.89      1300

Logistic Regression - No PCA
Accuracy: 82.31%
Confusion Matrix:
[[1014   47]
 [ 183   56]]
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.96      0.90      1061
           1       0.54      0.23      0.33       239

    accuracy                           0.82      1300
   macro avg       0.70      0.60      0.61      1300
weighted avg       0.79      0.82      0.79      1300

SVM - No PCA
Accuracy: 72.15%
Confusion Matrix:
[[727 334]
 [ 28 211]]
Classification Report:
              precision    rec

In [51]:
import copy

for name, model in models.items():
    # Train and evaluate with PCA
    if name == "Random Forest":
        model_pca = RandomForestClassifier(n_estimators=100, random_state=42)
    else:
        # Deep copy preserves model settings (like class_weight for SVM)
        model_pca = copy.deepcopy(model)

    model_pca.fit(X_train_pca, y_train)
    y_pred_pca = model_pca.predict(X_test_pca)
    acc_pca = accuracy_score(y_test, y_pred_pca)
    results.append((name, "Models With PCA", acc_pca))

    print(f"{name} - With PCA")
    print(f"Accuracy: {acc_pca:.2%}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred_pca))
    print("Classification Report:")
    print(classification_report(y_test, y_pred_pca, zero_division=0))

Random Forest - With PCA
Accuracy: 87.92%
Confusion Matrix:
[[1008   53]
 [ 104  135]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.95      0.93      1061
           1       0.72      0.56      0.63       239

    accuracy                           0.88      1300
   macro avg       0.81      0.76      0.78      1300
weighted avg       0.87      0.88      0.87      1300

Logistic Regression - With PCA
Accuracy: 81.54%
Confusion Matrix:
[[1013   48]
 [ 192   47]]
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      1061
           1       0.49      0.20      0.28       239

    accuracy                           0.82      1300
   macro avg       0.67      0.58      0.59      1300
weighted avg       0.78      0.82      0.78      1300

SVM - With PCA
Accuracy: 69.69%
Confusion Matrix:
[[698 363]
 [ 31 208]]
Classification Report:
              precision 

## Conclusion

In this supervised learning project, we aimed to classify wines into two quality categories based on their chemical properties:

- **Standard** (quality ≤ 6)
- **Premium** (quality > 6)

We trained and evaluated three machine learning models — Random Forest, Logistic Regression, and Support Vector Machine (SVM), using both the full feature set and PCA-reduced versions. Each model was assessed using accuracy, precision, recall, and F1-score.

---

### Model Comparison Summary

| Model                 | PCA Used | Accuracy (%) | Notes                                                                 |
|-----------------------|----------|---------------|-----------------------------------------------------------------------|
| **Random Forest**     | No       | **89.46%**    | Best overall performance with high accuracy and strong class balance |
| Random Forest         | Yes      | 87.92%        | Slight drop after PCA, but still very effective                      |
| Logistic Regression   | No       | 82.31%        | Performs well, but weaker in capturing Premium wines                 |
| Logistic Regression   | Yes      | 81.54%        | Slight decrease in performance with PCA                              |
| SVM                   | No       | 72.15%        | Struggles to distinguish Standard wines                               |
| SVM                   | Yes      | 69.69%        | Slightly worse with PCA; limited predictive power                    |

---

### Model Insights

-  The Random Forest model without PCA delivered the best results, achieving over **89% accuracy**, clearly surpassing the baseline threshold of 75%. It handled the binary classification task effectively, capturing key patterns in chemical features like alcohol and density.

- PCA helped reduce feature complexity and kept model performance fairly stable, but didn’t significantly improve results. Random Forest consistently outperformed the others, while Logistic Regression and SVM struggled more with accurately identifying Premium wines.

- PCA helped simplify the data, and all models still performed well. Random Forest stayed the most accurate at 88.62%, but Logistic Regression and SVM had trouble identifying high-quality wines after PCA.


### Summary

This project confirmed that wine quality can be reliably predicted using machine learning techniques applied to chemical data. Features such as alcohol content, density, and sulphates were especially helpful in predicting whether a wine is Standard or Premium.

### What Was Achieved:
- Merged and cleaned red and white wine datasets, then standardized the chemical features.
- Used Spark SQL to explore how key features relate to wine quality.
- Built and compared three classification models using both full features and PCA.
- Achieved strong performance using a Random Forest Classifier, validating the effectiveness of machine learning in wine quality prediction.